####  MLPClassifier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from ImageWork import *
from sklearn.ensemble import RandomForestClassifier
import cv2
from sklearn.preprocessing import StandardScaler  

In [2]:
def pobitovo_sravnenie_WaterMark(W1, W2 , total_bit=1024 ):
    t1 = W1==W2
    sum=t1.sum()
    return sum/total_bit*100

def psnr(W, Wr):
    e = (np.sum((W - Wr) ** 2)) / (len(W) * len(W))
    p = 10 * np.log10(255 ** 2 / e)
    return p
    

def psnrCV2(W, Wr):
    p=cv2.PSNR(W,Wr)
    return p


def one_WM(DataFrame , path_save , path_water_mark , clf):
    del DataFrame['Unnamed: 0']
    X = DataFrame.iloc[:,[1,2,3,4]].values  
    Y=DataFrame.iloc[:,[0]].values
    y_predict_image=clf.predict(X)
    y_predict_image[y_predict_image>0]=255
    y_predict_image=y_predict_image.reshape(32,32)
    img = Image.fromarray(y_predict_image.astype(np.uint8))
    img.show()
    img.save(path_save)
    W=WaterMarkLoader.load(path_water_mark)
    WR=WaterMarkLoader.load(path_save)
    print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))
    

def NO_AttackModel_Test(dataframe,path_save , title , clf, path_water_mark="Water Mark Image/WaterMarkRandom.jpg", index_image=1):
    X_dataframe = dataframe.iloc[:,[1,2,3,4]].values  
    Y_dataframe = dataframe.iloc[:,[0]].values
    X_train, X_test, y_train, y_test = train_test_split(X_dataframe, Y_dataframe, test_size=0.4, random_state=42)
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
    # apply same transformation to test data
    X_test = scaler.transform(X_test)  
    print(title)
    y_predict=clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    print(pd.DataFrame(cm))
    print()
    print(classification_report(y_test, y_predict))
    print()
    X_image_test = dataframe.iloc[:,[1,2,3,4]].values  
    i=index_image
    y=X_image_test[1024*i:1024*(i+1)]
    y_predict_image=clf.predict(y)
    y_predict_image[y_predict_image>0]=255
    y_predict_image=y_predict_image.reshape(32,32)
    img = Image.fromarray(y_predict_image.astype(np.uint8))
    #img.show()
    img.save(path_save)
    W=WaterMarkLoader.load(path_water_mark)
    WR=WaterMarkLoader.load(path_save)
    print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))


In [3]:
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack_Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
del No_Attack_frame['Unnamed: 0']
X = No_Attack_frame.iloc[:,[1,2,3,4]].values  
Y=No_Attack_frame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

In [8]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes = (10,10,10,10) , max_iter=200 ,early_stopping=True , verbose=True)
clf.fit(X_train, y_train)


d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.67687682
Validation score: 0.736779
Iteration 2, loss = 0.48956353
Validation score: 0.772686
Iteration 3, loss = 0.41472461
Validation score: 0.823317
Iteration 4, loss = 0.35264188
Validation score: 0.873197
Iteration 5, loss = 0.29046496
Validation score: 0.923077
Iteration 6, loss = 0.22978534
Validation score: 0.945162
Iteration 7, loss = 0.17928255
Validation score: 0.955379
Iteration 8, loss = 0.15323149
Validation score: 0.961388
Iteration 9, loss = 0.13501748
Validation score: 0.965745
Iteration 10, loss = 0.12319595
Validation score: 0.968450
Iteration 11, loss = 0.11234040
Validation score: 0.970252
Iteration 12, loss = 0.10530344
Validation score: 0.972206
Iteration 13, loss = 0.09851793
Validation score: 0.974159
Iteration 14, loss = 0.09305568
Validation score: 0.974609
Iteration 15, loss = 0.09039708
Validation score: 0.976262
Iteration 16, loss = 0.08911649
Validation score: 0.976863
Iteration 17, loss = 0.08402793
Validation score: 0.976112
Iterat

MLPClassifier(early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10),
              verbose=True)

In [9]:
y_predict=clf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))

       0     1
0  25880   410
1    276  9274
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     26290
           1       0.96      0.97      0.96      9550

    accuracy                           0.98     35840
   macro avg       0.97      0.98      0.98     35840
weighted avg       0.98      0.98      0.98     35840



In [5]:
from sklearn.model_selection import GridSearchCV
params = {"hidden_layer_sizes":[(4,7,4),(6,12,6),(8,8,8,8),(5,15,15,5)] }
grid = GridSearchCV(estimator=clf, param_grid=params, scoring = 'accuracy')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_params_)

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.78302950
Iteration 2, loss = 0.64951501
Iteration 3, loss = 0.61056790
Iteration 4, loss = 0.58123290
Iteration 5, loss = 0.55709117
Iteration 6, loss = 0.53102234
Iteration 7, loss = 0.50306112
Iteration 8, loss = 0.47167432
Iteration 9, loss = 0.44141483
Iteration 10, loss = 0.41542480
Iteration 11, loss = 0.39404824
Iteration 12, loss = 0.37941223
Iteration 13, loss = 0.36904901
Iteration 14, loss = 0.35958403
Iteration 15, loss = 0.35191061
Iteration 16, loss = 0.34494368
Iteration 17, loss = 0.34073772
Iteration 18, loss = 0.33770951
Iteration 19, loss = 0.33563705
Iteration 20, loss = 0.33406585
Iteration 21, loss = 0.33282172
Iteration 22, loss = 0.33122595
Iteration 23, loss = 0.33041726
Iteration 24, loss = 0.32921928
Iteration 25, loss = 0.32848866
Iteration 26, loss = 0.32755584
Iteration 27, loss = 0.32574839
Iteration 28, loss = 0.32518537
Iteration 29, loss = 0.32386884
Iteration 30, loss = 0.32242705
Iteration 31, loss = 0.32189987
Iteration 32, los

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.91758202
Iteration 2, loss = 0.65663904
Iteration 3, loss = 0.61963131
Iteration 4, loss = 0.60086387
Iteration 5, loss = 0.58564652
Iteration 6, loss = 0.56956901
Iteration 7, loss = 0.54910098
Iteration 8, loss = 0.52037004
Iteration 9, loss = 0.49198565
Iteration 10, loss = 0.46885036
Iteration 11, loss = 0.44988307
Iteration 12, loss = 0.43356175
Iteration 13, loss = 0.41849982
Iteration 14, loss = 0.40462016
Iteration 15, loss = 0.38729098
Iteration 16, loss = 0.37518009
Iteration 17, loss = 0.36908461
Iteration 18, loss = 0.36404621
Iteration 19, loss = 0.35995324
Iteration 20, loss = 0.35749775
Iteration 21, loss = 0.35488513
Iteration 22, loss = 0.35332688
Iteration 23, loss = 0.35043247
Iteration 24, loss = 0.35027039
Iteration 25, loss = 0.34816920
Iteration 26, loss = 0.34707979
Iteration 27, loss = 0.34620925
Iteration 28, loss = 0.34468941
Iteration 29, loss = 0.34341811
Iteration 30, loss = 0.34237851
Iteration 31, loss = 0.34171401
Iteration 32, los

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.73755471
Iteration 2, loss = 0.66959802
Iteration 3, loss = 0.64307481
Iteration 4, loss = 0.61254808
Iteration 5, loss = 0.58790395
Iteration 6, loss = 0.54474595
Iteration 7, loss = 0.50325863
Iteration 8, loss = 0.47532584
Iteration 9, loss = 0.46157481
Iteration 10, loss = 0.44927881
Iteration 11, loss = 0.43989477
Iteration 12, loss = 0.42536647
Iteration 13, loss = 0.41026957
Iteration 14, loss = 0.39145245
Iteration 15, loss = 0.36594977
Iteration 16, loss = 0.33331520
Iteration 17, loss = 0.31336759
Iteration 18, loss = 0.30171841
Iteration 19, loss = 0.28807129
Iteration 20, loss = 0.27845666
Iteration 21, loss = 0.27275388
Iteration 22, loss = 0.26377730
Iteration 23, loss = 0.25633959
Iteration 24, loss = 0.25013288
Iteration 25, loss = 0.24498021
Iteration 26, loss = 0.23875291
Iteration 27, loss = 0.23662104
Iteration 28, loss = 0.23488258
Iteration 29, loss = 0.23429673
Iteration 30, loss = 0.22859413
Iteration 31, loss = 0.22412089
Iteration 32, los

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.66928384
Iteration 2, loss = 0.57067748
Iteration 3, loss = 0.49152155
Iteration 4, loss = 0.39964693
Iteration 5, loss = 0.31647074
Iteration 6, loss = 0.27523175
Iteration 7, loss = 0.25412460
Iteration 8, loss = 0.24312067
Iteration 9, loss = 0.23358097
Iteration 10, loss = 0.22859558
Iteration 11, loss = 0.22423453
Iteration 12, loss = 0.22043111
Iteration 13, loss = 0.21907597
Iteration 14, loss = 0.21642719
Iteration 15, loss = 0.21311045
Iteration 16, loss = 0.21168316
Iteration 17, loss = 0.20820526
Iteration 18, loss = 0.20579227
Iteration 19, loss = 0.20333959
Iteration 20, loss = 0.19985938
Iteration 21, loss = 0.19679545
Iteration 22, loss = 0.19472862
Iteration 23, loss = 0.19174844
Iteration 24, loss = 0.18840188
Iteration 25, loss = 0.18589916
Iteration 26, loss = 0.18156683
Iteration 27, loss = 0.17767663
Iteration 28, loss = 0.17493126
Iteration 29, loss = 0.17234163
Iteration 30, loss = 0.17220143
Iteration 31, loss = 0.16901536
Iteration 32, los

d:\code\python\nir\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,11.821960,0.643912,0.003928,0.006053,"(4, 7, 4)","{'hidden_layer_sizes': (4, 7, 4)}",0.927809,0.897611,0.770282,0.824594,0.911508,0.866361,0.059609,4
1,12.527428,0.845753,0.012635,0.006323,"(6, 12, 6)","{'hidden_layer_sizes': (6, 12, 6)}",0.976187,0.968525,0.963642,0.943434,0.968299,0.964017,0.011050,3
2,14.899685,0.807141,0.010755,0.006318,"(8, 8, 8, 8)","{'hidden_layer_sizes': (8, 8, 8, 8)}",0.977539,0.971680,0.983023,0.968975,0.946289,0.969501,0.012580,2
3,16.358911,0.778196,0.012321,0.007329,"(5, 15, 15, 5)","{'hidden_layer_sizes': (5, 15, 15, 5)}",0.982197,0.981295,0.981746,0.975436,0.935021,0.971139,0.018226,1


In [ ]:
clf_no_attack = RandomForestClassifier (max_depth = 30, n_estimators = 50 ,verbose=True)
clf_no_attack.fit(X_train, y_train)

In [ ]:
y_predict=clf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {"n_estimators":(10,20,50,100), "max_depth":(3,5,8,16)}
grid = GridSearchCV(estimator=clf_no_attack, param_grid=params, scoring = 'accuracy')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
pd.DataFrame(grid.cv_results_)

In [ ]:
X_image_test = No_Attack_frame.iloc[:,[1,2,3,4]].values  
i=8
y=X_image_test[1024*i:1024*(i+1)]
y_predict_image=clf.predict(y)
y_predict_image[y_predict_image>0]=255
y_predict_image=y_predict_image.reshape(32,32)
img = Image.fromarray(y_predict_image.astype(np.uint8))
img.show()
img.save("W_R/WR1.tif")
W=WaterMarkLoader.load("Water Mark Image/WaterMarkRandom.jpg")
WR=WaterMarkLoader.load("W_R/WR1.tif")
print("побитовое сравнение = ", pobitovo_sravnenie_WaterMark(W,WR))

In [ ]:
path_averege= "feature_vec/AverageAttack.txt"
path_histogram="feature_vec/HistogramAttack.txt"
path_GammaCor=  "feature_vec/GammaCorrection.txt"
path_JPEG50= "feature_vec/JPEG50.txt"
path_median=  "feature_vec/medianAttack.txt"
path_saltpaper="feature_vec/SaltPaperAttack.txt"
path_snarpness= "feature_vec/Sharpness.txt"

In [ ]:
averege=pd.read_csv(path_averege) 
histogram=pd.read_csv(path_histogram)
GammaCor=pd.read_csv(path_GammaCor) 
JPEG50=pd.read_csv(path_JPEG50) 
median=pd.read_csv(path_median) 
saltpaper=pd.read_csv(path_saltpaper)
snarpness=pd.read_csv(path_snarpness) 
frames = [averege, histogram, GammaCor , JPEG50, median,saltpaper ,snarpness]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del saltpaper['Unnamed: 0']
del averege['Unnamed: 0']
del histogram['Unnamed: 0']
del GammaCor['Unnamed: 0']
del median['Unnamed: 0']
del JPEG50['Unnamed: 0']
del snarpness['Unnamed: 0']

DataFrame

### тестирую модель обученную на noattack подсовываю атакованные изображения

In [ ]:
NO_AttackModel_Test(saltpaper,"W_R/WR_SP.tif", "ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack,)

In [ ]:
Attack_Dota_SP = pd.read_csv('feature_vec/SaltPaperAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_SP ,"W_R/WR_SP_Dota.tif","Water Mark Image/waterMark3.jpg" , clf_no_attack )


In [ ]:
NO_AttackModel_Test(JPEG50,"W_R/WR_JPEG50.tif", "ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

In [ ]:
Attack_Dota_jpeg = pd.read_csv('feature_vec/JPEG50Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_jpeg ,"W_R/WR_jpeg_Dota.tif","Water Mark Image/waterMark3.jpg" ,clf_no_attack )

In [ ]:
NO_AttackModel_Test(averege,"W_R/WR_averege.tif", "ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

In [ ]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

In [ ]:
NO_AttackModel_Test(histogram,"W_R/WR_histogram.tif", "ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

In [ ]:
Attack_Dota_Histogram = pd.read_csv('feature_vec/HistogramAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Histogram ,"W_R/WR_Histogram_Dota.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

In [ ]:
NO_AttackModel_Test(GammaCor,"W_R/WR_GammaCor.tif", "ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

In [ ]:
Attack_Dota_GammaCorrection= pd.read_csv('feature_vec/GammaCorrectionCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_GammaCorrection ,"W_R/WR_GammaCorrection.tif","Water Mark Image/waterMark3.jpg",clf_no_attack )

In [ ]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

In [ ]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_no_attack )

In [ ]:
NO_AttackModel_Test(snarpness,"W_R/WR_snarpness.tif", "ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_no_attack)

In [ ]:
Attack_Cross_Sharpness= pd.read_csv('feature_vec/SharpnessCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_Sharpness ,"W_R/SharpnessCross.tif","Water Mark Image/waterMark2.jpg",clf_no_attack )

### тестирую модель обученную на attack image подсовываю атакованные изображения

In [ ]:
X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [ ]:
clf_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_attack.fit(X_train, y_train)

In [ ]:
y_predict=clf_attack.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(pd.DataFrame(cm))
print(classification_report(y_test, y_predict))

In [ ]:
NO_AttackModel_Test(saltpaper,"W_R/WR_SP.tif", "ПРОВЕРКА ДЛЯ АТАКИ СОЛЬ-ПЕРЕЦ. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Dota_SP = pd.read_csv('feature_vec/SaltPaperAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_SP ,"W_R/WR_SP_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )


In [ ]:
NO_AttackModel_Test(averege,"W_R/WR_averege.tif", "ПРОВЕРКА ДЛЯ АТАКИ averege - среднее. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )

In [ ]:
NO_AttackModel_Test(histogram,"W_R/WR_histogram.tif", "ПРОВЕРКА ДЛЯ АТАКИ histogram. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Dota_Histogram = pd.read_csv('feature_vec/HistogramAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Histogram ,"W_R/WR_Histogram_Dota.tif","Water Mark Image/waterMark3.jpg",clf_attack )

In [ ]:
NO_AttackModel_Test(GammaCor,"W_R/WR_GammaCor.tif", "ПРОВЕРКА ДЛЯ АТАКИ GammaCor. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Dota_GammaCorrection= pd.read_csv('feature_vec/GammaCorrectionCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_GammaCorrection ,"W_R/WR_GammaCorrection.tif","Water Mark Image/waterMark3.jpg",clf_attack )

In [ ]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_attack )

In [ ]:
NO_AttackModel_Test(snarpness,"W_R/WR_snarpness.tif", "ПРОВЕРКА ДЛЯ АТАКИ snarpness. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Cross_Sharpness= pd.read_csv('feature_vec/SharpnessCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_Sharpness ,"W_R/SharpnessCross.tif","Water Mark Image/waterMark2.jpg",clf_attack )

In [ ]:
NO_AttackModel_Test(JPEG50,"W_R/WR_JPEG50.tif", "ПРОВЕРКА ДЛЯ АТАКИ JPEG50. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_attack)

In [ ]:
Attack_Dota_jpeg = pd.read_csv('feature_vec/JPEG50Dota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_jpeg ,"W_R/WR_jpeg_Dota.tif","Water Mark Image/waterMark3.jpg" ,clf_attack )

#  Обучаю сеть на одной конкретной атаке и смотрю на результат

### Медианная фильтрация

In [ ]:
median=pd.read_csv(path_median) 
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
frames = [ median , No_Attack_frame ]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del median['Unnamed: 0']
DataFrame

In [ ]:
X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

In [ ]:
clf_specific_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_specific_attack.fit(X_train, y_train)

In [ ]:
NO_AttackModel_Test(median,"W_R/WR_median.tif", "ПРОВЕРКА ДЛЯ АТАКИ median. МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_specific_attack)

In [ ]:
Attack_Cross_medianAttack= pd.read_csv('feature_vec/medianAttackCross.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Cross_medianAttack ,"W_R/medianAttackCross.tif","Water Mark Image/waterMark2.jpg",clf_specific_attack )

### Среднее

In [ ]:
averege=pd.read_csv(path_averege) 
No_Attack_frame = pd.read_csv('feature_vec/NO_Attack.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
frames = [ averege , No_Attack_frame ]
DataFrame = pd.concat(frames)
del DataFrame['Unnamed: 0']
del averege['Unnamed: 0']
DataFrame

X = DataFrame.iloc[:,[1,2,3,4]].values  
Y=DataFrame.iloc[:,[0]].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=42)

clf_specific_attack = MLPClassifier(hidden_layer_sizes = (20,15,10,5) , max_iter=1000 ,early_stopping=True , verbose=True)
clf_specific_attack.fit(X_train, y_train)

In [ ]:
NO_AttackModel_Test(median,"W_R/WR_average.tif", "ПРОВЕРКА ДЛЯ АТАКИ average МОДЕЛЬ ОБУЧЕНА НА НЕАТАКОВАННЫХ ИЗОБРАЖЕНИЯХ",clf_specific_attack)

In [ ]:
Attack_Dota_Average = pd.read_csv('feature_vec/AverageAttackDota.txt') # ПАРАМЕТР НА ВХОД ФУНКЦИИ
one_WM(Attack_Dota_Average ,"W_R/WR_Average_Dota.tif","Water Mark Image/waterMark3.jpg",clf_specific_attack )